In [1]:
%%capture
!pip install skyfield

In [22]:
import pytz

from skyfield.api import EarthSatellite, load, wgs84, Topos
from skyfield.units import Angle

from datetime import datetime
from zoneinfo import ZoneInfo

In [2]:
TLE_LINES = {
    'QZS-1': """
1 37158U 10045A   24311.48020424  .00000052  00000-0  00000+0 0  9995
2 37158  43.4369 122.8701 0004606 100.9393 256.1246  0.87715209 51233
    """,
    'QZS-2': """
1 42738U 17028A   24301.73140229 -.00000252  00000-0  00000-0 0  9990
2 42738  40.3668 251.8541 0742904 270.9223 275.0143  1.00248773  1634
    """,
    'QZS-3': """
1 42917U 17048A   24311.53591010 -.00000333  00000-0  00000-0 0  9998
2 42917   0.0832 126.5799 0002306  94.6745 144.9438  1.00267858 26360
""",
    'QZS-4': """
1 42965U 17062A   24309.45584984 -.00000297  00000-0  00000-0 0  9996
2 42965  40.4472 351.0121 0751871 270.6196  85.9148  1.00263884 25905
""",
    'QZS-1R': """
1 49336U 21096A   24310.68799663 -.00000189  00000-0  00000-0 0  9996
2 49336  36.2396  88.3895 0748751 270.5487  70.1628  1.00274068 11107
"""
}

In [34]:
ts = load.timescale()
observer = wgs84.latlon(20.99365, 105.86950277777778)

for name, tle in TLE_LINES.items():
  lines = [line.strip() for line in tle.split('\n') if line.strip()]
  line1, line2 = lines[0], lines[1]

  satellite = EarthSatellite(line1, line2)

  t = ts.now()

  difference = satellite - observer
  topocentric = difference.at(t)

  alt, az, distance = topocentric.altaz()
  if alt.degrees > 0:
    print(f'{name} is above the horizon')

    print('Altitude:', alt.degrees)
    print('Azimuth:', az.degrees)
    print('Distance: {:.1f} km'.format(distance.km))
    print('')

QZS-2 is above the horizon
Altitude: 35.84835423240179
Azimuth: 116.99033608588653
Distance: 37501.2 km

QZS-3 is above the horizon
Altitude: 55.73617278552114
Azimuth: 132.71306661006543
Distance: 36728.9 km

QZS-4 is above the horizon
Altitude: 20.061813414626464
Azimuth: 158.591297408454
Distance: 36601.2 km

QZS-1R is above the horizon
Altitude: 58.567850341627505
Azimuth: 83.25636779856339
Distance: 38449.5 km



In [35]:
def get_t(hour, minute, second):
  ts = load.timescale()
  hanoi_time = datetime.now(ZoneInfo('Asia/Ho_Chi_Minh')).replace(hour=hour, minute=minute, second=second, microsecond=0)
  utc_time = hanoi_time.astimezone(ZoneInfo('UTC'))
  return ts.from_datetime(utc_time)

t0 = get_t(0, 0, 0)
t1 = get_t(23, 59, 59)

for name, tle in TLE_LINES.items():
  print(name)

  lines = [line.strip() for line in tle.split('\n') if line.strip()]
  line1, line2 = lines[0], lines[1]

  satellite = EarthSatellite(line1, line2)

  satellite_t, events = satellite.find_events(observer, t0, t1, altitude_degrees=30.0)
  event_names = 'rise above 30°', 'culminate', 'set below 30°'

  for ti, event in zip(satellite_t, events):
    event_name = event_names[event]

    local_timezone = pytz.timezone('Asia/Ho_Chi_Minh')
    local_dt = ti.astimezone(local_timezone)
    event_time = local_dt.strftime('%Y-%m-%dT%H:%M:%S')

    print(event_time, event_name)

  print('')

QZS-1

QZS-2
2024-11-07T03:02:50 set below 30°
2024-11-07T09:56:15 rise above 30°
2024-11-07T14:25:31 culminate
2024-11-07T22:40:23 culminate

QZS-3
2024-11-07T09:17:22 culminate

QZS-4
2024-11-07T05:15:02 culminate
2024-11-07T09:24:59 set below 30°
2024-11-07T16:37:01 rise above 30°
2024-11-07T20:58:33 culminate

QZS-1R
2024-11-07T04:00:32 culminate
2024-11-07T11:34:59 culminate
2024-11-07T16:28:45 set below 30°
2024-11-07T22:49:18 rise above 30°

